# An upper bound on packing-based lower bounds

Sein $I = (G, c)$ eine Instanz des Weighted F-free Edge Editing Problems. Das Ziel von Lower Bounds ist es, die minimalen Optimierungskosten $k^*$ möglichst gut von unten abzuschätzen. Für ein Packing $P$ gilt

$$
0 \le c(P) \le c(P^*) \le k^*
$$

wobei $P^*$ das Packing mit den maximalen Kosten ist.


In diesem Notebook möchten wir die maximalen Kosten, die man durch ein Packing erreichen kann, von oben abzuschätzen. D.h. wir definieren einen Wert $S := S(I)$, für den gilt

$$c(P^*) \le S$$

In [ ]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from pathlib import Path
import yaml
import subprocess
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt

In [ ]:
MULTIPLIER = 100
PERMUTATION = 0

In [ ]:
def calculate_upper_bound(path, *, multiplier: int = MULTIPLIER):
    """Upper bound on packing costs for F = C4P4"""
    
    out = subprocess.run([
        "../cmake-build-release/instance_packing_problem",
        "--F", "C4P4",
        "--multiplier", str(multiplier),
        "--permutation", str(PERMUTATION),
        path
    ], capture_output=True)
    d = yaml.safe_load(out.stdout)

    covered_edges = d["covered_edges"]
    covered_edges_costs = np.array(d["covered_edges_costs"])

    covered_edges_costs[::-1].sort()
    S = covered_edges_costs[2::3].sum()
    return S

Seien $e_1, e_2, \dots, e_m$ die Kanten die Teil eines verbotenen Subgraphen sind mit der Ordnung $c(e_i) \geq c(e_{i+1})$. Dann ist

$$S = e_3 + e_6 + \dots = \sum_{i=1} c(e_{3 i})$$

eine obere Schranke für Packing Kosten.

Für F = C4P4
* Jeder verbotene Subgraph belegt 3 Kanten
* D.h. ein Packing besteht aus maximal |E|/3 Subgraphen
* Die Kosten eines Subgraphen sind maximal die Kosten der minimaler Kosten seiner Kanten

In [ ]:
def calculate_upper_bound_df(dataset: str, *,
                             print_progress: bool = True, subset = None,
                             multiplier: int = MULTIPLIER):
    paths = Path(f"../data/{dataset}/").glob("*.graph")
    
    if print_progress:
        paths = tqdm(list(paths))

    l = []
    for path in paths:
        instance = path.name
        k_opt = np.nan
        if subset is None or instance in subset:
            k_opt = calculate_upper_bound(path, multiplier=multiplier)
        l.append([dataset, instance, multiplier, permutation, k_opt])
    df = pd.DataFrame(l, columns=["dataset", "instance", "multiplier", "permutation", "upper_bound"])
    return df

In [ ]:
solution_df[solution_df["instance"] == "bio-nr-3-size-16.graph"]

In [ ]:
def load_dataset_optimal_editing_cost(dataset: str, F: str = "C4P4", print_progress: bool = True, multiplier: int = MULTIPLIER):
    """Load ''*.solution.yaml' file."""
    paths = Path(f"../experiments/{F}/solutions/{dataset}/").glob(f"*.{multiplier}.solution.yaml")
    
    if print_progress:
        paths = tqdm(list(paths))
    
    def load_single(path):
        with path.open() as f:
            d = yaml.safe_load(f)
        return d
    
    ds = [load_single(path) for path in paths]
    df = json_normalize(ds)
    
    df[["dataset", "instance"]] = df["instance.name"].str.split("/", expand=True)[[1, 2]]
    df.rename(columns={
        "solution_cost": "optimal_editing_cost",
        "instance.multiplier": "multiplier",
        "instance.permutation": "permutation"}, inplace=True)
    df = df[["forbidden_subgraphs", "dataset", "instance", "multiplier", "permutation", "optimal_editing_cost"]]
    df.loc[df["optimal_editing_cost"] == -1, "optimal_editing_cost"] = np.nan
    return df

In [ ]:
def load_dataset_metadata(dataset: str):
    """Load ''*.metadata.yaml' file."""
    with open(f"../data/{dataset}/{dataset}.metadata.yaml") as f:
        meta_d = list(yaml.safe_load(f))
    df = json_normalize(meta_d)
    return df

In [ ]:
def load_lower_bound_benchmark(dataset, *, F="C4P4"):
    paths = Path(f"../experiments/{F}").glob(f"lb*/{dataset}.benchmarks.yaml")
    paths = tqdm(list(paths))

    ds = []
    for path in paths:
        with path.open() as f:
            d = list(yaml.safe_load_all(f))
            ds.extend(d)
    
    df = json_normalize(ds)
    
    df[["dataset", "instance"]] = df["instance"].str.split("/", expand=True)[[1, 2]]
    df["lower_bound_value"] = df["values"].str[0]
    df = df[["forbidden_subgraphs", "dataset", "instance", "multiplier", "permutation", "lower_bound_name", "lower_bound_value"]]
    
    return df

In [ ]:
def load_fpt_editing(dataset, *, F="C4P4"):
    paths = Path(f"../experiments/{F}").glob(f"fpt.timelimit=100.selector=MostAdjacentSubgraphs*Fixed/{dataset}.solutions.df.gzip")
    paths = list(paths)
    print(paths)

load_fpt_editing("bio")

# Execution

In [ ]:
lower_bound_df = load_lower_bound_benchmark("bio")

In [ ]:
solution_df = load_dataset_optimal_editing_cost("bio")

In [ ]:
solved = set(solution_df.loc[~solution_df["optimal_editing_cost"].isnull(), "instance"])
df = calculate_upper_bound_df("bio", subset=solved)

# Visualization

### First
* Compare lower bounds to upper bound
* Compare runtime of FPT-algorithm to upper bound
  * Hypotheses: When $k^*$ is noticably larger than the upper bound, the runtime increases strongly

### Second
* Plot $lb / m$ and a line at $\frac{1}{3}$
  * The LPRelaxation should be able to be larger than $\frac{1}{3}$
* Plot $k^* / S$ or $k^* / m$
  * Plot as Matrix

In [ ]:
solution_df = solution_df.sort_values("instance").reset_index(drop=True)
df = df.sort_values("instance").reset_index(drop=True)

df["optimal_editing_cost"] = solution_df["optimal_editing_cost"]

In [ ]:
for lb, lb_df in lower_bound_df.groupby("lower_bound_name"):
    print(lb, len(lb_df))

In [ ]:
lb_names = lower_bound_df["lower_bound_name"].unique()
fig, axes = plt.subplots(figsize=(len(lb_names) * 4, 3), ncols=len(lb_names))

for ax, lb_name in zip(axes, lb_names):
    lb_df = lower_bound_df[lower_bound_df["lower_bound_name"] == lb_name].sort_values("instance").reset_index(drop=True)
    x = lb_df["lower_bound_value"] / df["upper_bound"]

    ax.set_title(lb_name)
    ax.hist(x, bins=40)
    
plt.show()

In [ ]:
lb_df = lower_bound_df[lower_bound_df["lower_bound_name"] == "LPRelaxation"].sort_values("instance").reset_index(drop=True)
x = lb_df["lower_bound_value"] / df["upper_bound"]

lb_df[x > 1]

In [ ]:
df[df["upper_bound"] < df["optimal_editing_cost"]]

In [ ]:
lb_names = lower_bound_df["lower_bound_name"].unique()
fig, axes = plt.subplots(figsize=(len(lb_names) * 4, 3), ncols=len(lb_names))

for ax, lb_name in zip(axes, lb_names):
    lb_df = lower_bound_df[lower_bound_df["lower_bound_name"] == lb_name].sort_values("instance").reset_index(drop=True)
    x = lb_df["lower_bound_value"] / df["optimal_editing_cost"]

    ax.set_title(lb_name)
    ax.hist(x, bins=np.linspace(0, 1, 40))

plt.show()

In [ ]:
lb_names = lower_bound_df["lower_bound_name"].unique()

fig, axes = plt.subplots(figsize=(8, 4 * len(lb_names)), nrows=len(lb_names), sharey=True, sharex=True)


for ax, lb_name in zip(axes, lb_names):
    lb_df = lower_bound_df[lower_bound_df["lower_bound_name"] == lb_name].sort_values("instance").reset_index(drop=True)
    x = lb_df["lower_bound_value"] / df["upper_bound"]
    y = lb_df["lower_bound_value"] / df["optimal_editing_cost"]
    
    ax.set_xlabel("$lb / S$"); ax.set_ylabel("$lb / k_{opt}$")
    ax.set_xlim((-0.02, 2.5)); ax.set_ylim((-0.02, 1.02))
    ax.set_aspect("equal")

    ax.set_title(lb_name)
    ax.scatter(x, y, s=10, alpha=0.5)
    ax.axvline(1, c="k", ls="--", alpha=0.5)

fig.tight_layout()
plt.show()

In [ ]:
fpt_editing_df = pd.read_pickle("../experiments/C4P4/fpt.timelimit=100.selector=MostAdjacentSubgraphs.lower-bound=SortedGreedy.all=1.pre-mark=0.search-strategy=Fixed/bio.solutions.df.gzip")
fpt_editing_df.loc[fpt_editing_df["total_time"] == -1, "total_time"] = np.nan

In [ ]:
df[lb_df["lower_bound_value"] == df["upper_bound"]]

In [ ]:
lb_name = "SortedGreedy"

lb_df = lower_bound_df[lower_bound_df["lower_bound_name"] == lb_name].sort_values("instance").reset_index(drop=True)
edit_df = fpt_editing_df[fpt_editing_df["lower_bound"] == lb_name].sort_values("instance").reset_index(drop=True)



x = lb_df["lower_bound_value"] / df["upper_bound"]
y = edit_df["total_time"] / 10**9

fig, axes = plt.subplots(figsize=(12, 4), ncols=3)
plt.suptitle(lb_name, y=1.05)

axes[0].set_xscale("log"); axes[0].set_yscale("log")
axes[0].set_xlim((10**-6, 10**0.5)); axes[0].set_ylim((10**-5, 10**2.5))
axes[1].set_xlim((0, 1)); axes[1].set_ylim((0, 100))

for ax in axes[:2]:
    ax.set_xlabel("$lb / S$"); ax.set_ylabel("$t$")

axes[0].scatter(x + 10**-5, y, alpha=0.2)
axes[1].scatter(x, y, alpha=0.2)
axes[2].hist(x, bins=40)
fig.tight_layout()
plt.show()



x = lb_df["lower_bound_value"] / df["optimal_editing_cost"]
y = edit_df["total_time"] / 10**9

fig, ax = plt.subplots()

ax.set_yscale("log"); ax.set_ylim((10**-5, 10**2.5))
#ax.set_xlim((10**-7, 1)); ax.set_ylim((10**-5, 10**2))
ax.set_xlabel("$lb - / k_{opt}$"); ax.set_ylabel("$t$")

ax.scatter(x, y, alpha=0.2)
plt.show()

In [ ]:
x = lower_bound_df[lower_bound_df["lower_bound_name"] == "LocalSearch"].sort_values("instance").reset_index(drop=True)

df[x["lower_bound_value"] == df["optimal_editing_cost"]]